In [5]:
pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.4 MB/s eta 0:00:00


**DESAFIO #1**: Construya un script que genere de forma automática los datos de: departamentos, puestos de trabajo, y empleados

In [8]:
# Importamos las librerias necesarias para crear los datos
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import boto3

# Creamos los datos por departamento
departamentos_data = {
    "departamento_id": np.arange(1, 6),
    "departamento_nombre": ["Ventas", "Marketing", "I+D", "Recursos Humanos", "Contabilidad"]
}

# Creamos los datos por puesto de trabajo
puestos_data = {
    "puesto_id": np.arange(101, 106),
    "puesto_nombre": ["Gerente", "Cientifico de datos", "Desarrollador", "Asistente", "Especialista"]
}

# Creamos los datos de empleados teniendo en cuenta los departamentos y los puestos por id
def generar_empleados(num_empleados):
    empleados_data = {
        "empleado_id": np.arange(1001, 1001 + num_empleados),
        "nombre": [f"Empleado {i}" for i in range(1, num_empleados + 1)],
        "departamento_id": np.random.randint(1, 6, size=num_empleados),
        "puesto_id": np.random.randint(101, 106, size=num_empleados),
        "salario": np.random.randint(20000, 80000, size=num_empleados),
        "fecha_contratacion": [datetime(2023, np.random.randint(1, 13), np.random.randint(1, 29)) for _ in range(num_empleados)]
    }
    return empleados_data

# Teniendo en cuenta la funcion generada vamos a crear un total de 10 empleados
empleados_data = generar_empleados(10)

# almacenamos los datos generados en diferentes dataframes
departamentos_df = pd.DataFrame(departamentos_data)
puestos_df = pd.DataFrame(puestos_data)
empleados_df = pd.DataFrame(empleados_data)

print(departamentos_df)
print(puestos_df)
print(empleados_df)


   departamento_id departamento_nombre
0                1              Ventas
1                2           Marketing
2                3                 I+D
3                4    Recursos Humanos
4                5        Contabilidad
   puesto_id        puesto_nombre
0        101              Gerente
1        102  Cientifico de datos
2        103        Desarrollador
3        104            Asistente
4        105         Especialista
   empleado_id       nombre  departamento_id  puesto_id  salario  \
0         1001   Empleado 1                1        101    57540   
1         1002   Empleado 2                3        101    41193   
2         1003   Empleado 3                4        101    38344   
3         1004   Empleado 4                5        105    78268   
4         1005   Empleado 5                1        103    42413   
5         1006   Empleado 6                2        105    31984   
6         1007   Empleado 7                2        105    20429   
7         1008   E

**DESAFIO #2**: Guarde los datos simulados en archivos con formato CSV/Parquet. Explique el porqué de la escogencia del formato. No descarte usar la
capa gratuita de algún servicio de almacenamiento tipo cloud, será considerado un plus

**Respuesta:** Se escoje el formato csv ya que es un formato bastante estandar el cual se puede cargar y leer a multiples fuentes de datos, para este caso podemos cargar los archivos csv en blobs de azure o en buckets de s3 de amazon los cuales luego pueden ser procesados y cargados a bases de datos SQL.

In [7]:
# Exportar los DataFrames a archivos CSV
departamentos_df.to_csv('departamentos.csv', index=False)
puestos_df.to_csv('puestos.csv', index=False)
empleados_df.to_csv('empleados.csv', index=False)

**DESAFIO #3:** Implemente un proceso batch para migrar los datos a una base de datos SQL/NoSQL, o si lo desea, a un Datawarehouse o bucket analítico
de un Datalake. No descarte usar la capa gratuita de algún servicio de almacenamiento tipo cloud, será considerado un plus

RESPUESTA: Para este caso vamos a usar un bucket analitico para mantener la carga de los archivos, para este proceso no cuento con el servicio de s3 de AWS pero se dejara el codigo con el cual nos vamos a conectar a este bucket y vamos a realizar el cargue de los archivos a diferentes carpetas dentro del bucket, esto con el fin de definir un proceso en el cual se pueden estar generando multiples archivos y estos se puedan cargar en esta ruta, se deja el codigo con las variables creadas en donde solo tendremos que agregar los datos de la conexion al bucket de s3.

In [ ]:
# Configuramos los parametros de conexion a nuestro bucket en AWS
AWS_ACCESS_KEY_ID = 'TU_ACCESS_KEY_ID' # Access key generado en el bucket para la conexion
AWS_SECRET_ACCESS_KEY = 'TU_SECRET_ACCESS_KEY' #secret access key generado en el bucket para la conexion
REGION_NAME = 'TU_REGION_NAME'  #'us-east-1' por lo general es una buena region utilizada desde latinoamerica

# Realizamos la conexion de la parte del cliente utilizando la libreria instalada previamente la cual es boto3
s3_client = boto3.client('s3',
                         aws_access_key_id=AWS_ACCESS_KEY_ID,
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                         region_name=REGION_NAME)

# Definimos la ruta donde se van a almacenar los archivos en este caso creamos 3
bucket_departamentos = 's3://Proyecto/departamentos/'
bucket_puestos = 's3://Proyecto/puestos/'
bucket_empleados = 's3://Proyecto/empleados/'

# Rutas de los archivos CSV locales
departamentos_csv_path = 'departamentos.csv'
puestos_csv_path = 'puestos.csv'
empleados_csv_path = 'empleados.csv'

# Cargar archivos CSV a S3
s3_client.upload_file(departamentos_csv_path, bucket_departamentos, 'departamentos.csv')
s3_client.upload_file(puestos_csv_path, bucket_puestos, 'puestos.csv')
s3_client.upload_file(empleados_csv_path, bucket_empleados, 'empleados.csv')

print("Archivos CSV cargados exitosamente a Amazon S3.")

Para la visualizacion de los datos desde una base de datos, podemos usar el servicio de Athenas el cual es una base de datos enfocada a temas de datawarehouse en donde podremos visualizar los datos del bucket usando una tabla externa, si no queremos trabajar con la parte de aws podemos hacer el proceso con sql server estableciendo la conexion desde python y realizando el cargue de los archivos.

**DESAFIO #4: **Dependiendo si escoge una base de datos SQL/NoSQL, un Datawarehouse, o un Datalake, entonces desarrolle una view/query/report a
partir del modelo de datos.

En la parte de la base de datos ya teniendo cargada la informacion se puede crear la siguiente vista la cual nos permite validar cuantas personas se tienen en cada carga, esto es inicial ya que teniendo los datos podemos mejorarla para validar mas aspectos importantes en la vista directamente

In [ ]:
#Creacion de la vista desde el motor de base de datos
create view Personas_por_puesto as
select b.puesto_nombre, count(a.puesto_id) as Conteo_Puesto from [dbo].[empleados] a
left join [dbo].[puestos] b on a.puesto_id = b.puesto_id
group by b.puesto_nombre